In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier,\
      GradientBoostingClassifier, StackingClassifier, VotingClassifier
from xgboost import XGBClassifier
import json


## 数据处理部分

In [3]:
data = pd.read_pickle("data/data_processed_augmented_missing_threshold_2_outlier_std_threshold_3.pkl")
X_orig = data.drop(columns=['Number of nights in CITY', 'Purpose of visit to CITY']).to_numpy()
y = data['Purpose of visit to CITY'].to_numpy()

In [4]:
# ss = np.load('data/outliers_unscaled.npz')
# outliers = ss['outliers'].astype(int)
# rows_to_remove = outliers
# X_orig = np.delete(X_orig, rows_to_remove, axis=0)
# y = np.delete(y, rows_to_remove)

In [5]:
pca_50 = PCA(n_components= 50)  
X_pca_50 = pca_50.fit_transform(X_orig)

pca_100 = PCA(n_components= 100)  
X_pca_100 = pca_100.fit_transform(X_orig)

pca_200 = PCA(n_components= 200)  
X_pca_200 = pca_200.fit_transform(X_orig)

In [6]:
lda = LDA(n_components=min(len(np.unique(y)) - 1, X_orig.shape[1]))
X_lda = lda.fit_transform(X_orig, y)

In [ ]:
select_50 = SelectKBest(k=50)
X_selected_50 = select_50.fit_transform(X_orig, y)

select_100 = SelectKBest(k=100)
X_selected_100 = select_100.fit_transform(X_orig, y)

select_200 = SelectKBest(k=200)
X_selected_200 = select_200.fit_transform(X_orig, y)

In [7]:
tscv = TimeSeriesSplit(n_splits=5)

In [8]:
print(X_orig.shape)

(71214, 226)


In [9]:
print(y)

[3. 2. 1. ... 1. 1. 1.]


In [10]:
y = y.astype(int)
print(y)

[3 2 1 ... 1 1 1]


## Logistic Regression

In [ ]:
Logistic_Reg_params = {
    'C': [0.01, 0.1, 1.0, 5]
}
name = 'Logistic Regression'

feature_sets = {
    'Original': X_orig,
    'PCA_50': X_pca_50,
    'PCA_100': X_pca_100,
    'PCA_200': X_pca_200,
    'Select50Best': X_selected_50,
    'Select100Best': X_selected_100,
    'Select200Best': X_selected_200,
    'LDA': X_lda,
    'Scaled_Original': StandardScaler().fit_transform(X_orig),
    'Scaled_PCA_50': StandardScaler().fit_transform(X_pca_50),
    'Scaled_PCA_100': StandardScaler().fit_transform(X_pca_100),
    'Scaled_PCA_200': StandardScaler().fit_transform(X_pca_200),
    'Scaled_Select50Best': StandardScaler().fit_transform(X_selected_50),
    'Scaled_Select100Best': StandardScaler().fit_transform(X_selected_100),
    'Scaled_Select200Best': StandardScaler().fit_transform(X_selected_200),
    'Scaled_LDA': StandardScaler().fit_transform(X_lda),
}

# Dictionary to store results for all feature sets
results = {
    'model_name': name,
    'hyperparameters': Logistic_Reg_params,
    'feature_sets': {}
}

for feature_name, X in feature_sets.items():
    # Lists to store metrics and model parameters for each fold
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(X), desc=f'Processing {feature_name}')):
        X_train, X_test = X[train_idx], X[test_idx]
        y_clf_train, y_clf_test = y[train_idx], y[test_idx]
        
        # Define and train model with GridSearchCV
        model = LogisticRegression(max_iter=1000, solver='saga', n_jobs=20)
        grid = GridSearchCV(model, Logistic_Reg_params, cv=2, n_jobs=20)
        grid.fit(X_train, y_clf_train)
        model = grid.best_estimator_
        
        # Make predictions and compute metrics
        y_pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_clf_test, y_pred, average='weighted', zero_division=1)
        auc = roc_auc_score(y_clf_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        
        # Store fold metrics
        fold_result = {
            'fold': fold,
            'accuracy': float(accuracy_score(y_clf_test, y_pred)),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'auc': float(auc)
        }
        fold_metrics.append(fold_result)
                        
        # Print per-fold results
        print(f'{name}-{feature_name} - Fold {fold} - Accuracy: {fold_result["accuracy"]:.2f}, '
              f'Precision: {fold_result["precision"]:.2f}, Recall: {fold_result["recall"]:.2f}, '
              f'F1: {fold_result["f1"]:.2f}, AUC: {fold_result["auc"]:.2f}')
    
    # Compute average metrics across folds
    avg_metrics = {
        'avg_accuracy': float(np.mean([m['accuracy'] for m in fold_metrics])),
        'avg_precision': float(np.mean([m['precision'] for m in fold_metrics])),
        'avg_recall': float(np.mean([m['recall'] for m in fold_metrics])),
        'avg_f1': float(np.mean([m['f1'] for m in fold_metrics])),
        'avg_auc': float(np.mean([m['auc'] for m in fold_metrics]))
    }
    
    # Store results for this feature set
    results['feature_sets'][feature_name] = {
        'fold_metrics': fold_metrics,
        'average_metrics': avg_metrics
    }
    
    # Print average results
    print(f'\nAverage Performance Across Folds for {feature_name}:')
    print(f'{name}-{feature_name} - Accuracy: {avg_metrics["avg_accuracy"]:.2f}, '
          f'Precision: {avg_metrics["avg_precision"]:.2f}, Recall: {avg_metrics["avg_recall"]:.2f}, '
          f'F1: {avg_metrics["avg_f1"]:.2f}, AUC: {avg_metrics["avg_auc"]:.2f}\n')

# Save results to JSON file
with open(f'results/clf_aug/{name}_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results and model parameters have been saved to 'logistic_regression_results.json'")

Processing Original: 0it [00:00, ?it/s]d:\Anaconda\envs\mypytorch\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Processing Original: 1it [01:14, 74.75s/it]

Logistic Regression-Original - Fold 0 - Accuracy: 0.63, Precision: 0.52, Recall: 0.63, F1: 0.55, AUC: 0.54


d:\Anaconda\envs\mypytorch\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Processing Original: 2it [03:49, 122.07s/it]

Logistic Regression-Original - Fold 1 - Accuracy: 0.70, Precision: 0.70, Recall: 0.70, F1: 0.58, AUC: 0.58


d:\Anaconda\envs\mypytorch\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Processing Original: 3it [07:29, 166.73s/it]

...

Logistic Regression-Scaled_LDA - Fold 2 - Accuracy: 0.88, Precision: 0.87, Recall: 0.88, F1: 0.87, AUC: 0.95


Processing Scaled_LDA: 4it [00:00,  5.22it/s]

Logistic Regression-Scaled_LDA - Fold 3 - Accuracy: 0.88, Precision: 0.88, Recall: 0.88, F1: 0.88, AUC: 0.96


Processing Scaled_LDA: 5it [00:00,  5.13it/s]

Logistic Regression-Scaled_LDA - Fold 4 - Accuracy: 0.88, Precision: 0.87, Recall: 0.88, F1: 0.87, AUC: 0.95

Average Performance Across Folds for Scaled_LDA:
Logistic Regression-Scaled_LDA - Accuracy: 0.88, Precision: 0.87, Recall: 0.88, F1: 0.87, AUC: 0.95

Results and model parameters have been saved to 'logistic_regression_results.json'


## Decision Tree

In [ ]:
Decision_Tree_params = {
    'max_depth': [50, 100, 200]
    }
name = 'Decision Tree'

feature_sets = {
    'Original': X_orig,
    'PCA_50': X_pca_50,
    'PCA_100': X_pca_100,
    'PCA_200': X_pca_200,
    'Select50Best': X_selected_50,
    'Select100Best': X_selected_100,
    'Select200Best': X_selected_200,
    'LDA': X_lda,
    'Scaled_Original': StandardScaler().fit_transform(X_orig),
    'Scaled_PCA_50': StandardScaler().fit_transform(X_pca_50),
    'Scaled_PCA_100': StandardScaler().fit_transform(X_pca_100),
    'Scaled_PCA_200': StandardScaler().fit_transform(X_pca_200),
    'Scaled_Select50Best': StandardScaler().fit_transform(X_selected_50),
    'Scaled_Select100Best': StandardScaler().fit_transform(X_selected_100),
    'Scaled_Select200Best': StandardScaler().fit_transform(X_selected_200),
    'Scaled_LDA': StandardScaler().fit_transform(X_lda),
}

# Dictionary to store results for all feature sets
results = {
    'model_name': name,
    'hyperparameters': Decision_Tree_params,
    'feature_sets': {}
}

for feature_name, X in feature_sets.items():
    # Lists to store metrics and model parameters for each fold
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(X), desc=f'Processing {feature_name}')):
        X_train, X_test = X[train_idx], X[test_idx]
        y_clf_train, y_clf_test = y[train_idx], y[test_idx]
        
        # Define and train model with GridSearchCV
        model = DecisionTreeClassifier()
        grid = GridSearchCV(model, Decision_Tree_params, cv=2, n_jobs=20)
        grid.fit(X_train, y_clf_train)
        model = grid.best_estimator_
        
        # Make predictions and compute metrics
        y_pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_clf_test, y_pred, average='weighted', zero_division=1)
        auc = roc_auc_score(y_clf_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        
        # Store fold metrics
        fold_result = {
            'fold': fold,
            'accuracy': float(accuracy_score(y_clf_test, y_pred)),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'auc': float(auc)
        }
        fold_metrics.append(fold_result)
                        
        # Print per-fold results
        print(f'{name}-{feature_name} - Fold {fold} - Accuracy: {fold_result["accuracy"]:.2f}, '
              f'Precision: {fold_result["precision"]:.2f}, Recall: {fold_result["recall"]:.2f}, '
              f'F1: {fold_result["f1"]:.2f}, AUC: {fold_result["auc"]:.2f}')
    
    # Compute average metrics across folds
    avg_metrics = {
        'avg_accuracy': float(np.mean([m['accuracy'] for m in fold_metrics])),
        'avg_precision': float(np.mean([m['precision'] for m in fold_metrics])),
        'avg_recall': float(np.mean([m['recall'] for m in fold_metrics])),
        'avg_f1': float(np.mean([m['f1'] for m in fold_metrics])),
        'avg_auc': float(np.mean([m['auc'] for m in fold_metrics]))
    }
    
    # Store results for this feature set
    results['feature_sets'][feature_name] = {
        'fold_metrics': fold_metrics,
        'average_metrics': avg_metrics
    }
    
    # Print average results
    print(f'\nAverage Performance Across Folds for {feature_name}:')
    print(f'{name}-{feature_name} - Accuracy: {avg_metrics["avg_accuracy"]:.2f}, '
          f'Precision: {avg_metrics["avg_precision"]:.2f}, Recall: {avg_metrics["avg_recall"]:.2f}, '
          f'F1: {avg_metrics["avg_f1"]:.2f}, AUC: {avg_metrics["avg_auc"]:.2f}\n')

# Save results to JSON file
with open(f'results/clf_aug/{name}_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results and model parameters have been saved to 'logistic_regression_results.json'")

## SVC

In [ ]:
name = 'SVC'

feature_sets = {
    # 'Original': X_orig,
    # 'PCA_50': X_pca_50,
    # 'PCA_100': X_pca_100,
    # 'PCA_200': X_pca_200,
    # 'Select50Best': X_selected_50,
    # 'Select100Best': X_selected_100,
    # 'Select200Best': X_selected_200,
    # 'LDA': X_lda,
    # 'Scaled_Original': StandardScaler().fit_transform(X_orig),
    'Scaled_PCA_50': StandardScaler().fit_transform(X_pca_50),
    # 'Scaled_PCA_100': StandardScaler().fit_transform(X_pca_100),
    # 'Scaled_PCA_200': StandardScaler().fit_transform(X_pca_200),
    'Scaled_Select50Best': StandardScaler().fit_transform(X_selected_50),
    # 'Scaled_Select100Best': StandardScaler().fit_transform(X_selected_100),
    # 'Scaled_Select200Best': StandardScaler().fit_transform(X_selected_200),
    'Scaled_LDA': StandardScaler().fit_transform(X_lda),
}

# Dictionary to store results for all feature sets
results = {
    'model_name': name,
    'feature_sets': {}
}

for feature_name, X in feature_sets.items():
    # Lists to store metrics and model parameters for each fold
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(X), desc=f'Processing {feature_name}')):
        X_train, X_test = X[train_idx], X[test_idx]
        y_clf_train, y_clf_test = y[train_idx], y[test_idx]
        
        # Define and train model with GridSearchCV
        model = SVC(verbose=1, probability=True, C=0.1, kernel='rbf')
        model.fit(X_train, y_clf_train)
        
        # Make predictions and compute metrics
        y_pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_clf_test, y_pred, average='weighted', zero_division=1)
        auc = roc_auc_score(y_clf_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        
        # Store fold metrics
        fold_result = {
            'fold': fold,
            'accuracy': float(accuracy_score(y_clf_test, y_pred)),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'auc': float(auc)
        }
        fold_metrics.append(fold_result)
                        
        # Print per-fold results
        print(f'{name}-{feature_name} - Fold {fold} - Accuracy: {fold_result["accuracy"]:.2f}, '
              f'Precision: {fold_result["precision"]:.2f}, Recall: {fold_result["recall"]:.2f}, '
              f'F1: {fold_result["f1"]:.2f}, AUC: {fold_result["auc"]:.2f}')
    
    # Compute average metrics across folds
    avg_metrics = {
        'avg_accuracy': float(np.mean([m['accuracy'] for m in fold_metrics])),
        'avg_precision': float(np.mean([m['precision'] for m in fold_metrics])),
        'avg_recall': float(np.mean([m['recall'] for m in fold_metrics])),
        'avg_f1': float(np.mean([m['f1'] for m in fold_metrics])),
        'avg_auc': float(np.mean([m['auc'] for m in fold_metrics]))
    }
    
    # Store results for this feature set
    results['feature_sets'][feature_name] = {
        'fold_metrics': fold_metrics,
        'average_metrics': avg_metrics
    }
    
    # Print average results
    print(f'\nAverage Performance Across Folds for {feature_name}:')
    print(f'{name}-{feature_name} - Accuracy: {avg_metrics["avg_accuracy"]:.2f}, '
          f'Precision: {avg_metrics["avg_precision"]:.2f}, Recall: {avg_metrics["avg_recall"]:.2f}, '
          f'F1: {avg_metrics["avg_f1"]:.2f}, AUC: {avg_metrics["avg_auc"]:.2f}\n')

# Save results to JSON file
with open(f'results/clf_aug/{name}_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results and model parameters have been saved to 'logistic_regression_results.json'")

Processing Scaled_PCA_50: 0it [00:00, ?it/s]

[LibSVM]

Processing Scaled_PCA_50: 1it [00:26, 26.01s/it]

SVC-Scaled_PCA_50 - Fold 0 - Accuracy: 0.76, Precision: 0.79, Recall: 0.76, F1: 0.70, AUC: 0.83
[LibSVM]

Processing Scaled_PCA_50: 2it [01:54, 62.55s/it]

...

SVC-Scaled_LDA - Fold 2 - Accuracy: 0.88, Precision: 0.87, Recall: 0.88, F1: 0.87, AUC: 0.93
[LibSVM]

Processing Scaled_LDA: 4it [02:22, 45.34s/it]

SVC-Scaled_LDA - Fold 3 - Accuracy: 0.89, Precision: 0.88, Recall: 0.89, F1: 0.88, AUC: 0.91
[LibSVM]

Processing Scaled_LDA: 5it [04:13, 50.64s/it]

SVC-Scaled_LDA - Fold 4 - Accuracy: 0.88, Precision: 0.87, Recall: 0.88, F1: 0.87, AUC: 0.91

Average Performance Across Folds for Scaled_LDA:
SVC-Scaled_LDA - Accuracy: 0.88, Precision: 0.87, Recall: 0.88, F1: 0.87, AUC: 0.92

Results and model parameters have been saved to 'logistic_regression_results.json'


# 集成算法

## Random Forest

In [ ]:
rf_params = {
    'n_estimators': [50, 100], 
    'max_depth': [100, 200]
    }
name = 'Random Forest'

feature_sets = {
    'Original': X_orig,
    'PCA_50': X_pca_50,
    # 'PCA_100': X_pca_100,
    # 'PCA_200': X_pca_200,
    'Select50Best': X_selected_50,
    # 'Select100Best': X_selected_100,
    # 'Select200Best': X_selected_200,
    'LDA': X_lda,
    'Scaled_Original': StandardScaler().fit_transform(X_orig),
    'Scaled_PCA_50': StandardScaler().fit_transform(X_pca_50),
    # 'Scaled_PCA_100': StandardScaler().fit_transform(X_pca_100),
    # 'Scaled_PCA_200': StandardScaler().fit_transform(X_pca_200),
    'Scaled_Select50Best': StandardScaler().fit_transform(X_selected_50),
    # 'Scaled_Select100Best': StandardScaler().fit_transform(X_selected_100),
    # 'Scaled_Select200Best': StandardScaler().fit_transform(X_selected_200),
    'Scaled_LDA': StandardScaler().fit_transform(X_lda),
}

# Dictionary to store results for all feature sets
results = {
    'model_name': name,
    'hyperparameters': rf_params,
    'feature_sets': {}
}

for feature_name, X in feature_sets.items():
    # Lists to store metrics and model parameters for each fold
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(X), desc=f'Processing {feature_name}')):
        X_train, X_test = X[train_idx], X[test_idx]
        y_clf_train, y_clf_test = y[train_idx], y[test_idx]
        
        # Define and train model with GridSearchCV
        model = RandomForestClassifier()
        grid = GridSearchCV(model, rf_params, cv=2)
        grid.fit(X_train, y_clf_train)
        model = grid.best_estimator_
        
        # Make predictions and compute metrics
        y_pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_clf_test, y_pred, average='weighted', zero_division=1)
        auc = roc_auc_score(y_clf_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        
        # Store fold metrics
        fold_result = {
            'fold': fold,
            'accuracy': float(accuracy_score(y_clf_test, y_pred)),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'auc': float(auc)
        }
        fold_metrics.append(fold_result)
                        
        # Print per-fold results
        print(f'{name}-{feature_name} - Fold {fold} - Accuracy: {fold_result["accuracy"]:.2f}, '
              f'Precision: {fold_result["precision"]:.2f}, Recall: {fold_result["recall"]:.2f}, '
              f'F1: {fold_result["f1"]:.2f}, AUC: {fold_result["auc"]:.2f}')
    
    # Compute average metrics across folds
    avg_metrics = {
        'avg_accuracy': float(np.mean([m['accuracy'] for m in fold_metrics])),
        'avg_precision': float(np.mean([m['precision'] for m in fold_metrics])),
        'avg_recall': float(np.mean([m['recall'] for m in fold_metrics])),
        'avg_f1': float(np.mean([m['f1'] for m in fold_metrics])),
        'avg_auc': float(np.mean([m['auc'] for m in fold_metrics]))
    }
    
    # Store results for this feature set
    results['feature_sets'][feature_name] = {
        'fold_metrics': fold_metrics,
        'average_metrics': avg_metrics
    }
    
    # Print average results
    print(f'\nAverage Performance Across Folds for {feature_name}:')
    print(f'{name}-{feature_name} - Accuracy: {avg_metrics["avg_accuracy"]:.2f}, '
          f'Precision: {avg_metrics["avg_precision"]:.2f}, Recall: {avg_metrics["avg_recall"]:.2f}, '
          f'F1: {avg_metrics["avg_f1"]:.2f}, AUC: {avg_metrics["avg_auc"]:.2f}\n')

# Save results to JSON file
with open(f'results/clf_aug/{name}_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results and model parameters have been saved to 'logistic_regression_results.json'")

## Bagging Method

In [ ]:
name = 'Bagging'

feature_sets = {
    'Original': X_orig,
    'PCA_50': X_pca_50,
    'PCA_100': X_pca_100,
    'PCA_200': X_pca_200,
    'Select50Best': X_selected_50,
    'Select100Best': X_selected_100,
    'Select200Best': X_selected_200,
    'LDA': X_lda,
    'Scaled_Original': StandardScaler().fit_transform(X_orig),
    'Scaled_PCA_50': StandardScaler().fit_transform(X_pca_50),
    'Scaled_PCA_100': StandardScaler().fit_transform(X_pca_100),
    'Scaled_PCA_200': StandardScaler().fit_transform(X_pca_200),
    'Scaled_Select50Best': StandardScaler().fit_transform(X_selected_50),
    'Scaled_Select100Best': StandardScaler().fit_transform(X_selected_100),
    'Scaled_Select200Best': StandardScaler().fit_transform(X_selected_200),
    'Scaled_LDA': StandardScaler().fit_transform(X_lda),
}

# Dictionary to store results for all feature sets
results = {
    'model_name': name,
    'feature_sets': {}
}

for feature_name, X in feature_sets.items():
    # Lists to store metrics and model parameters for each fold
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(X), desc=f'Processing {feature_name}')):
        X_train, X_test = X[train_idx], X[test_idx]
        y_clf_train, y_clf_test = y[train_idx], y[test_idx]
        
        # Define and train model with GridSearchCV
        model = BaggingClassifier(estimator=LogisticRegression(C=0.1), n_estimators=10, random_state=42, n_jobs=20)
        model.fit(X_train, y_clf_train)
        
        # Make predictions and compute metrics
        y_pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_clf_test, y_pred, average='weighted', zero_division=1)
        auc = roc_auc_score(y_clf_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        
        # Store fold metrics
        fold_result = {
            'fold': fold,
            'accuracy': float(accuracy_score(y_clf_test, y_pred)),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'auc': float(auc)
        }
        fold_metrics.append(fold_result)
                        
        # Print per-fold results
        print(f'{name}-{feature_name} - Fold {fold} - Accuracy: {fold_result["accuracy"]:.2f}, '
              f'Precision: {fold_result["precision"]:.2f}, Recall: {fold_result["recall"]:.2f}, '
              f'F1: {fold_result["f1"]:.2f}, AUC: {fold_result["auc"]:.2f}')
    
    # Compute average metrics across folds
    avg_metrics = {
        'avg_accuracy': float(np.mean([m['accuracy'] for m in fold_metrics])),
        'avg_precision': float(np.mean([m['precision'] for m in fold_metrics])),
        'avg_recall': float(np.mean([m['recall'] for m in fold_metrics])),
        'avg_f1': float(np.mean([m['f1'] for m in fold_metrics])),
        'avg_auc': float(np.mean([m['auc'] for m in fold_metrics]))
    }
    
    # Store results for this feature set
    results['feature_sets'][feature_name] = {
        'fold_metrics': fold_metrics,
        'average_metrics': avg_metrics
    }
    
    # Print average results
    print(f'\nAverage Performance Across Folds for {feature_name}:')
    print(f'{name}-{feature_name} - Accuracy: {avg_metrics["avg_accuracy"]:.2f}, '
          f'Precision: {avg_metrics["avg_precision"]:.2f}, Recall: {avg_metrics["avg_recall"]:.2f}, '
          f'F1: {avg_metrics["avg_f1"]:.2f}, AUC: {avg_metrics["avg_auc"]:.2f}\n')

# Save results to JSON file
with open(f'results/clf_aug/{name}_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results and model parameters have been saved to 'logistic_regression_results.json'")

## Voting

In [11]:
name = 'Voting'

estimators = [
    ('dt', DecisionTreeClassifier(max_depth=200)),
    ('svc', SVC(probability=True)), 
    ('LogisticRegression', LogisticRegression(C=0.5))
]

feature_sets = {
    # 'Original': X_orig,
    # 'PCA_50': X_pca_50,
    # 'PCA_100': X_pca_100,
    # 'PCA_200': X_pca_200,
    # 'Select50Best': X_selected_50,
    # 'Select100Best': X_selected_100,
    # 'Select200Best': X_selected_200,
    # 'LDA': X_lda,
    # 'Scaled_Original': StandardScaler().fit_transform(X_orig),
    'Scaled_PCA_50': StandardScaler().fit_transform(X_pca_50),
    # 'Scaled_PCA_100': StandardScaler().fit_transform(X_pca_100),
    # 'Scaled_PCA_200': StandardScaler().fit_transform(X_pca_200),
    'Scaled_Select50Best': StandardScaler().fit_transform(X_selected_50),
    # 'Scaled_Select100Best': StandardScaler().fit_transform(X_selected_100),
    # 'Scaled_Select200Best': StandardScaler().fit_transform(X_selected_200),
    'Scaled_LDA': StandardScaler().fit_transform(X_lda),
}

# Dictionary to store results for all feature sets
results = {
    'model_name': name,
    'feature_sets': {}
}

for feature_name, X in feature_sets.items():
    # Lists to store metrics and model parameters for each fold
    fold_metrics = []
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(X), desc=f'Processing {feature_name}')):
        X_train, X_test = X[train_idx], X[test_idx]
        y_clf_train, y_clf_test = y[train_idx], y[test_idx]
        
        # Define and train model with GridSearchCV
        model = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)  # 'hard' for majority voting
        model.fit(X_train, y_clf_train)
        
        # Make predictions and compute metrics
        y_pred = model.predict(X_test)
        precision, recall, f1, _ = precision_recall_fscore_support(y_clf_test, y_pred, average='weighted', zero_division=1)
        auc = roc_auc_score(y_clf_test, model.predict_proba(X_test), multi_class='ovr', average='weighted')
        
        # Store fold metrics
        fold_result = {
            'fold': fold,
            'accuracy': float(accuracy_score(y_clf_test, y_pred)),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'auc': float(auc)
        }
        fold_metrics.append(fold_result)
                        
        # Print per-fold results
        print(f'{name}-{feature_name} - Fold {fold} - Accuracy: {fold_result["accuracy"]:.2f}, '
              f'Precision: {fold_result["precision"]:.2f}, Recall: {fold_result["recall"]:.2f}, '
              f'F1: {fold_result["f1"]:.2f}, AUC: {fold_result["auc"]:.2f}')
    
    # Compute average metrics across folds
    avg_metrics = {
        'avg_accuracy': float(np.mean([m['accuracy'] for m in fold_metrics])),
        'avg_precision': float(np.mean([m['precision'] for m in fold_metrics])),
        'avg_recall': float(np.mean([m['recall'] for m in fold_metrics])),
        'avg_f1': float(np.mean([m['f1'] for m in fold_metrics])),
        'avg_auc': float(np.mean([m['auc'] for m in fold_metrics]))
    }
    
    # Store results for this feature set
    results['feature_sets'][feature_name] = {
        'fold_metrics': fold_metrics,
        'average_metrics': avg_metrics
    }
    
    # Print average results
    print(f'\nAverage Performance Across Folds for {feature_name}:')
    print(f'{name}-{feature_name} - Accuracy: {avg_metrics["avg_accuracy"]:.2f}, '
          f'Precision: {avg_metrics["avg_precision"]:.2f}, Recall: {avg_metrics["avg_recall"]:.2f}, '
          f'F1: {avg_metrics["avg_f1"]:.2f}, AUC: {avg_metrics["avg_auc"]:.2f}\n')

# Save results to JSON file
with open(f'results/clf_aug/{name}_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("Results and model parameters have been saved to 'logistic_regression_results.json'")

Processing Scaled_PCA_50: 1it [00:37, 37.01s/it]

Voting-Scaled_PCA_50 - Fold 0 - Accuracy: 0.72, Precision: 0.66, Recall: 0.72, F1: 0.68, AUC: 0.68


Processing Scaled_PCA_50: 2it [03:03, 101.19s/it]

Voting-Scaled_PCA_50 - Fold 1 - Accuracy: 0.73, Precision: 0.67, Recall: 0.73, F1: 0.69, AUC: 0.71


Processing Scaled_PCA_50: 3it [09:25, 229.71s/it]

...

Voting-Scaled_LDA - Fold 2 - Accuracy: 0.73, Precision: 0.66, Recall: 0.73, F1: 0.68, AUC: 0.67


Processing Scaled_LDA: 4it [09:25, 181.38s/it]

Voting-Scaled_LDA - Fold 3 - Accuracy: 0.72, Precision: 0.66, Recall: 0.72, F1: 0.68, AUC: 0.68


Processing Scaled_LDA: 5it [17:54, 214.94s/it]

Voting-Scaled_LDA - Fold 4 - Accuracy: 0.90, Precision: 0.86, Recall: 0.90, F1: 0.87, AUC: 0.75

Average Performance Across Folds for Scaled_LDA:
Voting-Scaled_LDA - Accuracy: 0.76, Precision: 0.70, Recall: 0.76, F1: 0.71, AUC: 0.69

Results and model parameters have been saved to 'logistic_regression_results.json'
